In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import seaborn as sns

In [2]:
dfX = pd.read_csv('./nba/X_train.csv')
dfy = pd.read_csv('./nba/y_train.csv')
dfr = pd.concat([dfX, dfy], axis=1)
dfr.columns = [s.lower() for s in dfr.columns]
df = pd.DataFrame()
df['fgm'] = dfr['fgm']
# df = dfr.copy()

In [3]:
dfr.describe()

,period,game_clock,shot_clock,dribbles,touch_time,shot_dist,pts_type,close_def_dist,fgm
count,52399.000000,52399.000000,52399.000000,52399.000000,52399.000000,52399.000000,52399.000000,52399.000000,52399.000000
mean,2.468444,351.650356,11.839358,2.026126,2.756631,13.529272,2.264146,4.117823,0.456268
std,1.136656,207.296328,6.172164,3.504288,3.098802,8.877446,0.440882,2.741868,0.498089
min,1.000000,0.000000,0.000000,0.000000,-163.600000,0.000000,2.000000,0.000000,0.000000
25%,1.000000,173.000000,7.500000,0.000000,0.900000,4.700000,2.000000,2.300000,0.000000
50%,2.000000,352.000000,11.900000,1.000000,1.600000,13.500000,2.000000,3.700000,0.000000
75%,3.000000,531.000000,16.300000,2.000000,3.700000,22.500000,3.000000,5.300000,1.000000
max,7.000000,720.000000,24.000000,30.000000,24.500000,46.300000,3.000000,53.200000,1.000000


In [4]:
dfr.corr()

,period,game_clock,shot_clock,dribbles,touch_time,shot_dist,pts_type,close_def_dist,fgm
period,1.000000,-0.027850,-0.032828,0.056460,0.046538,0.023997,0.041254,-0.011817,-0.009255
game_clock,-0.027850,1.000000,0.099661,-0.118390,-0.104398,-0.018371,-0.049029,0.006048,0.014278
shot_clock,-0.032828,0.099661,1.000000,-0.095571,-0.138361,-0.173521,-0.052573,0.021318,0.103239
dribbles,0.056460,-0.118390,-0.095571,1.000000,0.906443,-0.094191,-0.175923,-0.156581,-0.040758
touch_time,0.046538,-0.104398,-0.138361,0.906443,1.000000,-0.095077,-0.183437,-0.167082,-0.050196
shot_dist,0.023997,-0.018371,-0.173521,-0.094191,-0.095077,1.000000,0.744060,0.529047,-0.192289
pts_type,0.041254,-0.049029,-0.052573,-0.175923,-0.183437,0.744060,1.000000,0.424253,-0.122123
close_def_dist,-0.011817,0.006048,0.021318,-0.156581,-0.167082,0.529047,0.424253,1.000000,-0.002323
fgm,-0.009255,0.014278,0.103239,-0.040758,-0.050196,-0.192289,-0.122123,-0.002323,1.000000


In [5]:
df['fast_shot'] = (dfr['dribbles'] == 0) | (dfr['shot_clock'] > 22)
df['shot_dist'] = (dfr['shot_dist'] / 5).astype(int)

In [6]:
df.sample(10)

,fgm,fast_shot,shot_dist
33038,0,True,3
5801,1,True,0
44828,0,True,4
49215,0,True,5
10828,1,True,0
31721,0,False,3
5232,1,True,4
5403,0,False,3
48952,0,True,3
28940,0,False,5


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

xs = df.drop(['fgm'], axis=1).astype(float).values
ys = df['fgm'].astype(float).values.squeeze()
xt, xv, yt, yv = train_test_split(xs, ys, test_size=0.2, random_state=50)

std = StandardScaler().fit(xt)
xt_std = std.transform(xt)
xv_std = std.transform(xv)

print(xt.shape)
print(xv.shape)

# pca = PCA(n_components=5).fit(xt_std)
# xt_pca = pca.transform(xt_std)
# xv_pca = pca.transform(xv_std)

(41919, 2)
(10480, 2)


In [8]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, n_jobs=2)
rf.fit(xt, yt)
print('train: {:.3f}'.format(rf.score(xt, yt)))
print('valid: {:.3f}'.format(rf.score(xv, yv)))

train: 0.608
valid: 0.624


In [9]:
from sklearn.linear_model import RidgeClassifier
clf = RidgeClassifier()
clf.fit(xt_std, yt)
print('train: {:.3f}'.format(clf.score(xt_std, yt)))
print('valid: {:.3f}'.format(clf.score(xv_std, yv)))

train: 0.605
valid: 0.620


In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
reg = LinearRegression()
reg.fit(xt_std, yt)
print('train: {:.3f}'.format(accuracy_score(np.where(reg.predict(xt_std) < 0.5, 0, 1), yt)))
print('valid: {:.3f}'.format(accuracy_score(np.where(reg.predict(xv_std) < 0.5, 0, 1), yv)))

train: 0.605
valid: 0.620
